In [1]:
import numpy as np
from scipy.stats import norm, t
from scipy import linalg
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.pyplot as plt

In [2]:
###### Define parameters for Homogeneous portfolio
M     = 10      #number of Monte Carlo simulations in the default time modelling
N     = 10      #number of obligors on the basket default swap (basket size)
NN    = 4000    #number of points used in simulation of the random variables
h     = 0.25    #default intensity for all obligors (assumed deterministic)
rho   = 0.8     #correlation between obligors
df    = 1       #degree of freedom
n     = 1       #number to default in the basket default swap eg n=1 for first to default swap
TC    = 2.8820  #parameter value for the Clayton family (TC, TG and TF are in line with Kendall Tau's equivalence =0.49363)
TG    = 2.4410  #parameter value for the Gumbel family
TF    = 7.6762  #parameter value for the Frank family 

m= np.array([0,0]) #m : array_like, mean of random variable, length determines dimension of random variable
S= np.array([[1,rho],[rho,1]]) #S : array_like, square array of covariance  matrix

#Other parameters: r= deterministic risk-free interest rate, R= recovery rate of entities, dt= frequency payment dates (in units
#of years), T= maturity time,  m= time step for the premium payments, and mm= no of sub-timesteps within each dtfor integration




# RANDOM VARIABLES GENERATIONS WITH DIFFERENT COPULA MODELS

In [ ]:
#########################################################
#1.Gaussian
#########################################################
rv1=np.random.multivariate_normal(m, S, NN)
G=norm.cdf(rv1,0,1)
u = G[:,0]
v = G[:,1]
#plt.plot(u,v, '.')
#plt.show()
fig, ax = plt.subplots(figsize=(6,5)) # create a new figure with a default 111 subplot
ax.plot(u,v,'.')
axins = zoomed_inset_axes(ax, 2.5, loc=2) # zoom-factor: 2.5, location: best
axins.plot(u,v,'.')
axins.set_xlim(0.8, 1.0) # apply the x-limits
axins.set_ylim(0.8, 1.0) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
mark_inset(ax, axins, loc1=1, loc2=4, fc="none", lw=3,ec='b',zorder=3)
plt.draw()
plt.show()

#########################################################
#2.Student t
#########################################################
def multivariate_t_rvs(m, S, df=np.inf, NN=5000):
    if df == np.inf:
        x = 1.
    else:
        x = np.random.chisquare(df, NN)/df
    z = np.random.multivariate_normal(np.zeros(len(m)),S,(NN,))
    z1= m + z/np.sqrt(x)[:,None]
    return z1
rv2 = multivariate_t_rvs(m,S,df,NN)
V1=t.cdf(rv2,df)
u2=V1[:,0]
v2=V1[:,1]
#plt.plot(u2,v2,'.')
#plt.show()
fig, ax = plt.subplots(figsize=(6,5)) # create a new figure with a default 111 subplot
ax.plot(u2,v2,'.')
ax.set_xticks(np.arange(0.0, 1.1, 0.2))
ax.set_yticks(np.arange(0.0, 1.1, 0.2))

axins = zoomed_inset_axes(ax, 2.5, loc=2) # zoom-factor: 2.5, location: best
axins.plot(u2,v2,'.')
axins.set_xlim(0.8, 1.0) # apply the x-limits
axins.set_ylim(0.8, 1.0) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
mark_inset(ax, axins, loc1=1, loc2=4, fc="none", lw=3,ec='b',zorder=3)
plt.draw()
plt.show()


#########################################################
#3.Clayton
#########################################################
q  = np.random.rand(NN,1)
u3 = np.random.rand(NN,1)
v3 = ((q**(-TC/(1+TC))-1)*u3**(-TC)+1)**(-1/TC)    
#plt.plot(u3,v3, '.')
#plt.show() 
fig, ax = plt.subplots(figsize=(6,5)) # create a new figure with a default 111 subplot
ax.plot(u3,v3,'.')
axins = zoomed_inset_axes(ax, 2.5, loc=1) # zoom-factor: 2.5, location: best
axins.plot(u3,v3,'.')
axins.set_xlim(0.0, 0.2) # apply the x-limits
axins.set_ylim(0.0, 0.2) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", lw=3,ec='b',zorder=3)
plt.draw()
plt.show()  


#########################################################
#4.Gumbel
#########################################################
ru4 = np.random.rand(NN,1)
rv4 = np.random.rand(NN,1)
X = np.linspace(0,1,NN+2)[1:-1]
Kc = [w*(1-np.log(w)/TG) for w in X]
rv4=Kc
u4 = [np.exp(ru4[i]**(1/TG) * np.log(X[i])) for i in range(NN)]
v4 = [np.exp((1-ru4[i])**(1/TG) * np.log(X[i])) for i in range(NN)]
#plt.plot(u4,v4, '.')
#plt.show()
fig, ax = plt.subplots(figsize=(6,5)) # create a new figure with a default 111 subplot
ax.plot(u4,v4,'.')
axins = zoomed_inset_axes(ax, 2.5, loc=2) # zoom-factor: 2.5, location: best
axins.plot(u4,v4,'.')
axins.set_xlim(0.8, 1.00) # apply the x-limits
axins.set_ylim(0.8, 1.00) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
mark_inset(ax, axins, loc1=1, loc2=4, fc="none", lw=3,ec='b',zorder=3)
plt.draw()
plt.show()


#########################################################
#5.Frank
#########################################################
q1 = np.random.rand(NN,1)
u5 = np.random.rand(NN,1)
v5 = -1/TF*np.log(1+ (q1*(1-np.exp(-TF)))/(q1*(np.exp(-TF*u5)-1)-np.exp(-TF*u5)))
#plt.plot(u5,v5,'.')
#plt.show() 
fig, ax = plt.subplots(figsize=(6,5)) # create a new figure with a default 111 subplot
ax.plot(u5,v5,'.')
axins = zoomed_inset_axes(ax, 2.5, loc=2) # zoom-factor: 2.5, location: best
axins.plot(u5,v5,'.')
axins.set_xlim(0.8, 1.0) # apply the x-limits
axins.set_ylim(0.8, 1.0) # apply the y-limits
plt.yticks(visible=False)
plt.xticks(visible=False)
mark_inset(ax, axins, loc1=1, loc2=4, fc="none", lw=3,ec='b',zorder=3)
plt.draw()
plt.show()

# DEFAULT TIME MODELLING USING DIFFERENT COPULA MODELS

In [4]:
#########################################################
#1.Gaussian
#########################################################

mu    = np.zeros((N))
CM    = np.ones((N,N))*rho + (1-rho)*np.identity(N) #correlation matrix
rv    = np.random.multivariate_normal(mu, CM, M)
Tau_g = -1/h * (np.log(norm.cdf(rv,0,1)))

#########################################################
#2.Student t
#########################################################

CMa  = np.identity(N)-np.identity(N)*rho + np.ones((N,N))*rho  #correlation matrix
A    = linalg.cholesky(CMa)
rv   = np.random.randn(M, N)
y    = np.matmul(rv,A)
zz   = np.sqrt(df / np.random.chisquare(df,(1,N)))
x    = np.tile(zz,(M,1))*y
uu   = t.cdf(x,df)
Tau_S= -np.log(uu)/np.tile(h,(M,N)) 

#########################################################
#3.Clayton
#########################################################

U     = np.random.rand(M,N) #generates uniformly distributed random variables
X     = np.random.gamma(1/TC,1, (M, 1)) # generates gamma random variables, shape=theta, scale para=1, (p,1)=px1 matrix
U1    = -np.log(U)
X1    = np.matlib.repmat(X,1,N)
v     = [U1[i]/X1[i] for i in range(M)]
U     = [(1+v[i])**(-1/TC) for i in range(M)]
Tau_C = -1/h *(np.log(np.ones((M,N))-U))

#########################################################
#4.Gumbel
#########################################################

U2    = np.random.rand(M,N) #generates uniformly distributed random variables
alpha = 1/TG
beta  = 1
gamma = (np.cos(np.pi/(2*TG)))**TG
delta = 0
K     = np.random.uniform(-0.5*np.pi, 0.5*np.pi, (M,1)) #random variable in (-pi/2, pi/2)
W     = np.random.exponential(1, (M,1)) #exponential distributed rv with mean 1, independent of K

eta   = np.arctan(beta*np.tan(np.pi*alpha/2))/alpha
###for alpha!=1
z1    = (np.sin(alpha*(eta+K)))/((np.cos(alpha*eta)*np.cos(K))**(1/alpha))
z2    = (np.cos(alpha*eta+(alpha-1)*K)/W)**((1-alpha)/alpha)
Z     =z1*z2
#for alpha == 1.0:
#Z    = 2/np.pi*((2/np.pi+beta*K)*np.tan(K)-beta*np.log((np.pi/2*W*np.cos(K))/(np.pi/2 + beta*K)))
#XX   = gamma*Z + (delta + beta*2/np.pi*gamma*np.log(gamma)
XX    = gamma*Z + delta
U11   = -np.log(U2)
X11   = np.matlib.repmat(XX,1,N)
v1    = [U11[i]/X11[i] for i in range(M)]
U2    = [np.exp(-v1[i]**(1/TG)) for i in range(M)]
Tau_G = -1/h*(np.log(np.ones((M,N))-U2))

#########################################################
#5.Frank
#########################################################

U3    = np.random.rand(M,N) #generates uniformly distributed random variables
X3    = np.random.logseries((1-np.exp(-TF)), (M,1))
U31   = -np.log(U3)
X3    = np.matlib.repmat(X3,1,N)
v3    = [U31[i]/X3[i] for i in range(M)]
U3    = [-1/TF*np.log(1+np.exp(-v3[i])*(np.exp(-TF)-1)) for i in range(M)]
Tau_F = -1/h *(np.log(np.ones((M,N))-U3))



# SENSITIVITY ANALYSIS WRT RHO & THETA; LAMBDA; TIME

In [ ]:
########################################
#1. n2D BCDS using Gaussian copula model 
########################################

def BDS1(N, h, n, T, rho, r, R, M, dt, mm):
    
    m = int(T/dt)
    dt1 = dt/mm
    
    ########Simulation of default time via one-factor Gaussian copula###########
    mu=np.zeros((N))
    CM= np.ones((N,N))*rho + (1-rho)*np.eye(N) #covariance matrix
    RV=np.random.multivariate_normal(mu, CM, M)
    tau=-1/h * (np.log(norm.cdf(RV,0,1)))
    
    ######## Calculating the expected value of default leg
    DL=0
    Tau = np.sort(tau,1)
    Tau = Tau[:,n-1] 
    DL = 1/M*np.sum((1-R)*np.exp(-r *Tau)*(Tau <T))
    
    ######### Calulating the expected value of premium leg    
    t = dt
    PL = 0 
    for i in range(1,m+1):
        DB = np.exp(-r*t) #the discount bond at time t
        SP = 1-np.sum(np.sum(tau<t, 1)>=n, 0)/np.size(tau, axis=0) # survival prob of n2D until each indexing period
        PL=PL+ dt*DB*SP
        t = t + T/m
    
    ######## Calculating the expected value of accrued premium
    AP = 0
    t=0
    for i in range(1,m+1):
        q=t
        for j in range(1,mm+1):
            PD = 1/M * np.sum((Tau > (q)) & (Tau <= (q + dt1))) #prob of nth default between j and j+1
            q=q + dt1/2
            DB1=np.exp(-r*q)  #the discount bond at time q
            AP=AP + (q-t)*DB1*PD 
            
    spread1 = 1/(PL + AP)*10000*DL
    return spread1
#print(BDS1(10, 0.03, 1, 5, 0.5, 0.04,0.45, 50000, 0.25, 10))
 

########################################## 
#2. n2D BCDS using Student t copula model 
##########################################

def BDS2(N, df, h, n, T, rho, r, R, M, dt, mm):
    
    m = int(T/dt)
    dt1 = dt/mm
    
    ########Simulation of default time via one-factor Student t copula###########
    CorMat = np.identity(N)-np.identity(N)*rho + np.ones((N,N))*rho
    A=linalg.cholesky(CorMat)
    z = np.random.randn(M, N)
    s=np.random.chisquare(df,(1,N))
    y=np.matmul(z,A)
    zz=np.sqrt(df)/np.sqrt(s)
    x=np.tile(zz,(M,1))*y
    uu=t.cdf(x,df)
    tau=-np.log(uu)/np.tile(h,(M,N))
    
    ######## Calculating the expected value of default leg
    DL=0
    Tau = np.sort(tau,1)
    Tau = Tau[:,n-1] 
    DL = 1/M*np.sum((1-R)*np.exp(-r *Tau)*(Tau <T))
    
    ######### Calulating the expected value of premium leg    
    tt = dt
    PL = 0 
    for i in range(1,m+1):
        DB = np.exp(-r*tt) #the discount bond at time tt
        SP = 1-np.sum(np.sum(tau<tt, 1)>=n, 0)/np.size(tau, axis=0) # survival prob of n2D until each indexing period
        PL=PL+ dt*DB*SP
        tt = tt + T/m
    
    ######## Calculating the expected value of accrued premium
    AP = 0
    tt=0
    for i in range(1,m+1):
        q=tt
        for j in range(1,mm+1):
            PD = 1/M * np.sum((Tau > (q)) & (Tau <= (q + dt1))) #prob of nth default between j and j+1
            q=q + dt1/2
            DB1=np.exp(-r*q)  #the discount bond at time q
            AP=AP + (q-tt)*DB1*PD 
            
    spread2 = 1/(PL + AP)*10000*DL
    return spread2

######################################## 
#3. n2D BCDS using Clayton copula model 
########################################

def BDS3(N, Theta, h, n, T, r, R, M, dt, mm):
    
    m = int(T/dt)
    dt1 = dt/mm
    
    ####simulation of default times via clayton copula
    U  = np.random.rand(M,N) #generates uniformly distributed random variables
    X  = np.random.gamma(1/Theta,1, (M, 1)) # generates gamma random variables, shape=theta, scale para=1, (p,1)=px1 matrix
    U1 = -np.log(U)
    X1 = np.matlib.repmat(X,1,N)
    v  = [U1[i]/X1[i] for i in range(M)]
    U  = [(1+v[i])**(-1/Theta) for i in range(M)]
    tau= -(1/h)*(np.log(np.ones((M,N))-U))
    
    ######## Calculating the expected value of default leg
    DL=0
    Tau = np.sort(tau,1)
    Tau = Tau[:,n-1] 
    DL = 1/M*np.sum((1-R)*np.exp(-r *Tau)*(Tau <T))
    
    ######### Calulating the expected value of premium leg    
    tt = dt
    PL = 0 
    for i in range(1,m+1):
        DB = np.exp(-r*tt) #the discount bond at time tt
        SP = 1-np.sum(np.sum(tau<tt, 1)>=n, 0)/np.size(tau, axis=0) # survival prob of n2D until each indexing period
        PL=PL+ dt*DB*SP
        tt = tt + T/m
    
    ######## Calculating the expected value of accrued premium
    AP = 0
    tt=0
    for i in range(1,m+1):
        q=tt
        for j in range(1,mm+1):
            PD = 1/M * np.sum((Tau > (q)) & (Tau <= (q + dt1))) #prob of nth default between j and j+1
            q=q + dt1/2
            DB1=np.exp(-r*q)  #the discount bond at time q
            AP=AP + (q-tt)*DB1*PD 
            
    spread3 = 1/(PL + AP)*10000*DL
    return spread3

#######################################
#4. n2D BCDS using Gumbel copula model 
#######################################

def BDS4(N, Theta, h, n, T, r, R, M, dt, mm):
    
    m = int(T/dt)
    dt1 = dt/mm
    
    ####simulation of default times via Gumbel copula
    alpha = 1/Theta
    beta = 1
    gamma = (np.cos(np.pi/(2*Theta)))**Theta
    delt = 0
    K = np.random.uniform(-0.5*np.pi, 0.5*np.pi, (M,1)) #random variable in (-pi/2, pi/2)
    W = np.random.exponential(1, (M,1)) #exponential distributed rv with mean 1, independent of K

    eta= np.arctan(beta*np.tan(np.pi*alpha*0.5))/alpha
    z1 = (np.sin(alpha*(eta+K)))/((np.cos(alpha*eta)*np.cos(K))**(1/alpha))
    z2 = (np.cos(alpha*eta+(alpha-1)*K)/W)**((1-alpha)/alpha)
    Z=z1*z2
    XX = gamma*Z + delt
    
    U2 = np.random.rand(M,N) #generates uniformly distributed random variables
    U11   = -np.log(U2)
    X11   = np.matlib.repmat(XX,1,N)
    v1    = [U11[i]/X11[i] for i in range(M)]
    U2    = [np.exp(-v1[i]**(1/Theta)) for i in range(M)]
    tau   = -1/h *(np.log(np.ones((M,N))-U2))
    
    ######## Calculating the expected value of default leg
    DL=0
    Tau = np.sort(tau,1)
    Tau = Tau[:,n-1] 
    DL = 1/M*np.sum((1-R)*np.exp(-r *Tau)*(Tau <T))
    
    ######### Calulating the expected value of premium leg    
    tt = dt
    PL = 0 
    for i in range(1,m+1):
        DB = np.exp(-r*tt) #the discount bond at time tt
        SP = 1-np.sum(np.sum(tau<tt, 1)>=n, 0)/np.size(tau, axis=0) # survival prob of n2D until each indexing period
        PL=PL+ dt*DB*SP
        tt = tt + T/m
    
    ######## Calculating the expected value of accrued premium
    AP = 0
    tt=0
    for i in range(1,m+1):
        q=tt
        for j in range(1,mm+1):
            PD = 1/M * np.sum((Tau > (q)) & (Tau <= (q + dt1))) #prob of nth default between j and j+1
            q=q + dt1/2
            DB1=np.exp(-r*q)  #the discount bond at time q
            AP=AP + (q-tt)*DB1*PD 
            
    spread4 = 1/(PL + AP)*10000*DL
    return spread4

######################################
#5. n2D BCDS using Frank copula model 
######################################

def BDS5(N, Theta, h, n, T, r, R, M, dt, mm):
    
    m = int(T/dt)
    dt1 = dt/mm
    
    ####simulation of default times via Frank copula
    U3    = np.random.rand(M,N) #generates uniformly distributed random variables
    X3    = np.random.logseries((1-np.exp(-Theta)), (M,1))
    U31   = -np.log(U3)
    X3    = np.matlib.repmat(X3,1,N)
    v3    = [U31[i]/X3[i] for i in range(M)]
    U3    = [-1/Theta*np.log(1+np.exp(-v3[i])*(np.exp(-Theta)-1)) for i in range(M)]
    tau   = -1/h *(np.log(np.ones((M,N))-U3))
    
    ######## Calculating the expected value of default leg
    DL=0
    Tau = np.sort(tau,1)
    Tau = Tau[:,n-1] 
    DL = 1/M*np.sum((1-R)*np.exp(-r *Tau)*(Tau <T))
    
    ######### Calulating the expected value of premium leg    
    tt = dt
    PL = 0 
    for i in range(1,m+1):
        DB = np.exp(-r*tt) #the discount bond at time tt
        SP = 1-np.sum(np.sum(tau<tt, 1)>=n, 0)/np.size(tau, axis=0) # survival prob of n2D until each indexing period
        PL=PL+ dt*DB*SP
        tt = tt + T/m
    
    ######## Calculating the expected value of accrued premium
    AP = 0
    tt=0
    for i in range(1,m+1):
        q=tt
        for j in range(1,mm+1):
            PD = 1/M * np.sum((Tau > (q)) & (Tau <= (q + dt1))) #prob of nth default between j and j+1
            q=q + dt1/2
            DB1=np.exp(-r*q)  #the discount bond at time q
            AP=AP + (q-tt)*DB1*PD 
            
    spread5 = 1/(PL + AP)*10000*DL
    return spread5

##########Plotting (eg: Gaussian wrt to rho)######### 
if __name__ == "__main__":
    
    V1 = []
    V2 = []
    V3 = []
    N=10
    R=0.45
    T=5
    r=0.05
    M=50000
    dt=0.25
    rho1 = 0.35
    rho2 = 0.5
    rho3 = 0.7
    mm=10
    h=0.03
    for n in range(1,11):
        V1.append(BDS1(N, h, n, T, rho1, r, R, M, dt, mm))
        V2.append(BDS1(N, h, n, T, rho2, r, R, M, dt, mm))
        V3.append(BDS1(N, h, n, T, rho3, r, R, M, dt, mm))
        
    
plt.plot(range(1,11),V1,'r--', marker='o', label=r"$\rho=0.35$")
plt.plot(range(1,11),V2,'b--', marker='D', label=r"$\rho=0.5$")
plt.plot(range(1,11),V3,'g--', marker='s', label=r"$\rho=0.7$")
plt.xticks(np.arange(1, 11, 1.0))
plt.yticks(np.arange(0, 1001, 200.0))

plt.ylabel("n2D swap premuim", fontsize = 10)
plt.xlabel("n", fontsize = 10)
plt.legend(loc = 'best')
plt.show()
